In [1]:
import os
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

/Users/yujian/.pyenv/versions/3.10.1/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: dlopen(/Users/yujian/.pyenv/versions/3.10.1/lib/python3.10/site-packages/torchvision/image.so, 0x0006): Symbol not found: __ZN2at4_ops19empty_memory_format4callEN3c108ArrayRefINS2_6SymIntEEENS2_8optionalINS2_10ScalarTypeEEENS6_INS2_6LayoutEEENS6_INS2_6DeviceEEENS6_IbEENS6_INS2_12MemoryFormatEEE
  Referenced from: <0B637046-A38B-3A5C-80C6-E847C27DCCD5> /Users/yujian/.pyenv/versions/3.10.1/lib/python3.10/site-packages/torchvision/image.so
  Expected in:     <BB02660F-1D5B-3388-B48B-486877D726F6> /Users/yujian/.pyenv/versions/3.10.1/lib/python3.10/site-packages/torch/lib/libtorch_cpu.dylib
  warn(f"Failed to load image Python extension: {e}")


In [74]:
batchsize=50
training_data = datasets.FashionMNIST(root="../fashion_mnist", train=True, transform=transforms.ToTensor())
test_data = datasets.FashionMNIST(root="../fashion_mnist", train=False, transform=transforms.ToTensor())

train_dataloader = DataLoader(training_data, batch_size=batchsize)
test_dataloader = DataLoader(test_data, batch_size=batchsize)

In [67]:
training_data[0][0].size()

torch.Size([1, 28, 28])

Using: Fashion MNIST Dataset

60000 points

Each image is a 28x28 image

Tensor is of size 1 x 28 x 28

https://medium.com/@nutanbhogendrasharma/pytorch-recurrent-neural-networks-with-mnist-dataset-2195033b540f

https://stackoverflow.com/questions/44130851/simple-lstm-in-pytorch-with-sequential-module 

https://towardsdatascience.com/building-rnn-lstm-and-gru-for-time-series-using-pytorch-a46e5b094e7b

In [81]:
class NeuralNet(nn.Module):
    def __init__(self):
        super(NeuralNet, self).__init__()
        # create the layers
        self.flatten = nn.Flatten()
        self.num_layers = 2
        self.hidden_size = 10
        self.rnn_layer = nn.RNN(input_size=784, hidden_size=self.hidden_size, num_layers=self.num_layers)
        self.output_layer = nn.Linear(self.hidden_size, 10)
    
    def forward(self, X):
        X = self.flatten(X)
        # Passing in the input and hidden state into the model and  obtaining outputs
        out, hidden = self.rnn_layer(X)  # out: tensor of shape (batch_size, seq_length, hidden_size)
        # Can reshape the outputs such that it can be fit into the fully connected layer
        # out = self.output_layer(out[:, -1, :])
        out = self.output_layer(out)
        return out

model = NeuralNet()
learning_rate = 0.05
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [82]:
print(model)

NeuralNet(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (rnn_layer): RNN(784, 10, num_layers=2)
  (output_layer): Linear(in_features=10, out_features=10, bias=True)
)


In [83]:
# input = torch.stack([training_data[x][0] for x in range(0, 64)])
# model(input)

In [84]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        # reshape images
        # print(X.shape)
        X = X.reshape(-1, 28, 28)
        # print(X.shape)

        # predictions
        pred = model(X)
        loss = loss_fn(pred, y)

        # backprop
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch%100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"Current loss: {loss:>7f}, [{current:>5d}/{size:>5d}]")
            
def test_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            # reshape images
            X = X.reshape(-1, 28, 28)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    
    test_loss /= num_batches
    correct /= size
    print(f"Test Error:\n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f}\n")
    return 100*correct

In [85]:
def optimize_loop(epoch_list, accuracy_list, model, epochs, _optimizer=optimizer):
    for t in range(epochs):
        print(f"Epoch {t+1}\n")
        train_loop(train_dataloader, model, loss_fn, optimizer = _optimizer)
        accuracy = test_loop(test_dataloader, model, loss_fn, optimizer = _optimizer)
        epoch_list.append(t)
        accuracy_list.append(accuracy)
    print("Done!")

In [86]:
plot_x = [] # epoch
plot_y = [] # accuracy
optimize_loop(plot_x, plot_y, model, 10)

Epoch 1

Current loss: 2.310715, [    0/60000]
Current loss: 1.914559, [ 5000/60000]
Current loss: 1.382793, [10000/60000]
Current loss: 1.173555, [15000/60000]
Current loss: 1.082419, [20000/60000]
Current loss: 0.828579, [25000/60000]
Current loss: 0.923467, [30000/60000]
Current loss: 0.945968, [35000/60000]
Current loss: 0.849306, [40000/60000]
Current loss: 0.747513, [45000/60000]
Current loss: 0.745054, [50000/60000]
Current loss: 0.815328, [55000/60000]
Test Error:
 Accuracy: 74.3%, Avg loss: 0.708102

Epoch 2

Current loss: 0.599909, [    0/60000]
Current loss: 0.631169, [ 5000/60000]
Current loss: 0.576635, [10000/60000]
Current loss: 0.623970, [15000/60000]
Current loss: 0.502193, [20000/60000]
Current loss: 0.451544, [25000/60000]
Current loss: 0.713053, [30000/60000]
Current loss: 0.588562, [35000/60000]
Current loss: 0.667683, [40000/60000]
Current loss: 0.700588, [45000/60000]
Current loss: 0.631217, [50000/60000]
Current loss: 0.661902, [55000/60000]
Test Error:
 Accurac